In [1]:
import numpy as np
import matplotlib.pyplot as plt
import astropy.coordinates as coord
import astropy.units as u
from astropy import time

from poliastro.bodies import Earth
from poliastro.twobody import Orbit
import poliastro

ISS data and maneouver ephemeris https://spaceflight.nasa.gov/realdata/sightings/SSapplications/Post/JavaSSOP/orbit/ISS/SVPOST.html

More info about ISS maneouvers and how to read ephemeris. http://iss-transit.sourceforge.net/technical.html


Google Groups, how to convert from ECI (Earth-Centered Inertial) to lat-lon-height using astropy https://groups.google.com/forum/#!topic/astropy-dev/AIdMCZykFtw


In [2]:
Epoch_time = time.Time("2019-07-05 11:10:16") #19186.46546460   =   yrday.fracday

Inclination = 51.6422 * u.deg
RA_of_node = 270.5895  * u.deg
Eccentricity = 0.0007281  * u.one
Arg_of_perigee = 113.6717  * u.deg
Mean_anomaly = 246.5199  * u.deg
Rev_Per_Day = (15.50989459  / u.day).to(1/u.s)
semi_major_Axis = Earth.k**(1/3) / (2 * np.pi * Rev_Per_Day)**(2/3)  



In [3]:
yday = 186.46546460 * u.day
time.Time("2018-12-31 00:00") + yday



<Time object: scale='utc' format='iso' value=2019-07-05 11:10:16.141>

In [4]:
Epoch_time

<Time object: scale='utc' format='iso' value=2019-07-05 11:10:16.000>

In [5]:
Earth.k

<<class 'astropy.constants.constant.Constant'> name='Geocentric gravitational constant' value=398600441800000.0 uncertainty=800000.0 unit='m3 / s2' reference='IAU 2009 system of astronomical constants'>

In [6]:
semi_major_Axis

<Quantity 6791972.89119395 m>

In [7]:
(semi_major_Axis - Earth.R).to(u.km)

<Quantity 413.83629119 km>

In [8]:
ZAYRA = Orbit.from_classical(Earth, 
                             semi_major_Axis, 
                             Eccentricity, 
                             Inclination, 
                             RA_of_node,
                             Arg_of_perigee,
                             Mean_anomaly,
                             Epoch_time)

In [37]:
Tnow = time.Time.now()
DeltaT = Tnow - Epoch_time


ISSnow = ZAYRA.propagate( DeltaT )

print(ISSnow)

ISS_gcrs = coord.GCRS(
    coord.CartesianRepresentation(x=ISSnow.r[0], y=ISSnow.r[1], z=ISSnow.r[2]),
    obstime=time.Time.now())

ISS_itrs = ISS_gcrs.transform_to(coord.ITRS(obstime=Tnow))
ISS_el = coord.EarthLocation(ISS_itrs.x, ISS_itrs.y, ISS_itrs.z)
print(ISS_el)

lon, lat, height = ISS_el.to_geodetic()

print(lon.to(u.degree), lat.to(u.degree), height )


lon, lat, height 

6787 x 6797 km x 51.6 deg orbit around Earth (♁)
(-4843.947670942176, 492.23019976518526, -4739.395724937) km
174d11m51.5732s -44d24m29.2073s 426.99502909527297 km


(<Longitude 174.19765923 deg>,
 <Latitude -44.40811314 deg>,
 <Quantity 426.9950291 km>)